In [59]:
import torch
from model.model1 import Net
import pandas as pd
import random
import copy

In [60]:
# Load Training dataset
df = pd.read_csv('data/cleaned_train.csv')
df = df.sample(frac=1).reset_index(drop=True)

In [61]:
# Divide between train and test data
train = df[:800]
test = df[800:]
# Divide between x,y 
train_x = torch.tensor(train.iloc[:,3:].values).float()
train_y = torch.tensor(train.iloc[:,2].values).float()
test_x = torch.tensor(test.iloc[:,3:].values).float()
test_y = torch.tensor(test.iloc[:,2].values).float()


In [62]:
# Create the network
net = Net(inputs=17,outputs=1,dropout_prob=0.05)
# Move all to gpu
if torch.cuda.is_available():
   net = net.to("cuda:0")
   train_x = train_x.to("cuda:0")
   train_y = train_y.to("cuda:0")
   test_x = test_x.to("cuda:0")
   test_y = test_y.to("cuda:0")



In [70]:
# Prepare the network for training
epochs = 1000
batch_size = 10
# Initialize loss function
loss_fn = torch.nn.BCELoss()
# Initialize optimizer
optm = torch.optim.Adam(params=net.parameters(),lr=1e-2)
scheduler1 = torch.optim.lr_scheduler.LinearLR(optm,1,0.1,epochs)


In [64]:
# Divide tensors into batch sizes
train_x = train_x.split(batch_size);
train_y = train_y.split(batch_size);

In [71]:
losses = []
accuracies = []
num_batches = 50
test_y = test_y.reshape((test_y.shape[0],1))
max_accuracy = 0
best_model = copy.deepcopy(net.state_dict())
for epoch in range(epochs):
    net.train()
    for i in range(num_batches):
        indx = random.randint(0,len(train_x)-1)
        x = train_x[indx]
        y = train_y[indx]
        net_out = net.forward(x,x.size()[0])
        #print(all(i == net_out[0] for i in net_out))
        loss = loss_fn.forward(net_out,y.reshape(y.size()[0],1))
        optm.zero_grad()
        loss.backward()
        optm.step()
    scheduler1.step()
    with torch.no_grad():
        net.eval()
        net_out = net.forward(test_x,test_x.size()[0])
        loss = loss_fn.forward(net_out,test_y.reshape(test_y.size()[0],1))
        predictions = net_out.round()
        accuracy = 1- (predictions - test_y).abs().sum()/test_y.size()[0]
        if accuracy > max_accuracy:
            best_model = copy.deepcopy(net.state_dict())
            max_accuracy = accuracy
        losses.append(loss)
        accuracies.append(accuracies)
        print(f'EPOCH: {epoch}, Loss: {loss}, Accuracy: {accuracy}')


EPOCH: 0, Loss: 0.4952022135257721, Accuracy: 0.7692307829856873
EPOCH: 1, Loss: 0.4941532015800476, Accuracy: 0.7692307829856873
EPOCH: 2, Loss: 0.49300670623779297, Accuracy: 0.7692307829856873
EPOCH: 3, Loss: 0.49201831221580505, Accuracy: 0.7692307829856873
EPOCH: 4, Loss: 0.490939736366272, Accuracy: 0.7692307829856873
EPOCH: 5, Loss: 0.4900515377521515, Accuracy: 0.7692307829856873
EPOCH: 6, Loss: 0.48919105529785156, Accuracy: 0.7692307829856873
EPOCH: 7, Loss: 0.4882494807243347, Accuracy: 0.7692307829856873
EPOCH: 8, Loss: 0.4874338209629059, Accuracy: 0.7692307829856873
EPOCH: 9, Loss: 0.4867309331893921, Accuracy: 0.7692307829856873
EPOCH: 10, Loss: 0.48616519570350647, Accuracy: 0.7802197933197021
EPOCH: 11, Loss: 0.48568421602249146, Accuracy: 0.7802197933197021
EPOCH: 12, Loss: 0.4850236773490906, Accuracy: 0.7802197933197021
EPOCH: 13, Loss: 0.4845559298992157, Accuracy: 0.7802197933197021
EPOCH: 14, Loss: 0.4839252531528473, Accuracy: 0.7692307829856873
EPOCH: 15, Loss:

KeyboardInterrupt: 

In [44]:
torch.save(best_model,'saved_models/best_model.pt')